In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 33.7 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=2689c4db9911ffb18b0498b8d175efdcee891b33c8438b088a1c4f6e95edfaa6
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
requests 2.23.0 requires urllib3!

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Download 100%.


In [2]:
! signate download --competition-id=406

sample_submission.csv

train.csv

test_data.zip

train_data.zip


Download completed.


In [3]:
! unzip -q train_data.zip 
! unzip -q test_data.zip

In [4]:
! ls

__MACOSX     sample_submission.csv  test_data.zip  train_data
sample_data  test_data		    train.csv	   train_data.zip


In [5]:
!pip install -q omegaconf timm pytorch-metric-learning[with-hooks] umap-learn record-keeper tensorboard

     |████████████████████████████████| 79 kB 2.1 MB/s 
     |████████████████████████████████| 549 kB 60.3 MB/s 
     |████████████████████████████████| 111 kB 63.4 MB/s 
     |████████████████████████████████| 88 kB 8.5 MB/s 
     |████████████████████████████████| 117 kB 73.7 MB/s 
     |████████████████████████████████| 182 kB 70.4 MB/s 
     |████████████████████████████████| 1.1 MB 57.3 MB/s 
     |████████████████████████████████| 127 kB 53.5 MB/s 
     |████████████████████████████████| 85.5 MB 82 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
signate 0.9.9 requires urllib3>=1.26.7, but you have urllib3 1.25.11 which is incompatible.


# Training

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
import umap
import pytorch_metric_learning
import pytorch_metric_learning.utils.logging_presets as LP
from pytorch_metric_learning import miners, samplers, testers, trainers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [7]:
import os
import random
import numpy as np
import torch


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(0)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Config

In [9]:
train_master_dir = './train.csv'
img_dir = '/content/train_data'
OUTPUT_SIZE = 1024
EMBEDDING_SIZE = 512
log_dir = 'log'
tensorboard_dir = 'tensorboard'
model_dir = 'model'
patience=10
batch_size = 8
n_worker = 2
epochs = 10

Data

In [10]:
train_master = pd.read_csv(train_master_dir)

image_name_list = train_master['id'].values
label_list = train_master['target'].values

In [11]:
x_train, x_val, y_train, y_val = train_test_split(image_name_list, label_list, test_size=0.25, stratify=label_list, random_state=42)

Transformation

In [12]:
from torchvision import transforms


class Transforms():

    def __init__(self):
        
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(300),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            'val': transforms.Compose([
                transforms.Resize(300),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            'test': transforms.Compose([
                transforms.Resize(300),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                ]),
            }
    
    def __call__(self, phase, img):
        """
        Parameters
        ----------
        phase : 'train' or 'val' or 'test'
        """
        return self.data_transform[phase](img)

Dataset

In [13]:
import os
from PIL import Image
from torch.utils.data import Dataset


class DefectDataset(Dataset):
    def __init__(self, image_name_list, label_list, img_dir, transform=None, phase=None):
        self.image_name_list = image_name_list
        self.label_list = label_list
        self.img_dir = img_dir
        self.phase = phase
        self.transform = transform

    def __len__(self):
        return len(self.image_name_list)

    def __getitem__(self, index):
        image_path = os.path.join(self.img_dir, self.image_name_list[index])
        image = Image.open(image_path)
        image = self.transform(self.phase, image)
        label = self.label_list[index]
        
        return image, label

In [14]:
from omegaconf import DictConfig


def get_dataset(x_train, y_train, x_val, y_val):
    train_dataset = DefectDataset(
        image_name_list=x_train, 
        label_list=y_train,
        img_dir=img_dir,
        transform=Transforms(),
        phase='train'
    )
    val_dataset = DefectDataset(
        image_name_list=x_val, 
        label_list=y_val,
        img_dir=img_dir,
        transform=Transforms(),
        phase='val'
    )

    return train_dataset, val_dataset

In [15]:
train_dataset, val_dataset = get_dataset(x_train, y_train, x_val, y_val)

DataLoader

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

Model

In [17]:
import torch
import torch.nn as nn
import timm
import pytorch_metric_learning
from pytorch_metric_learning.utils import common_functions


class ConvnextBase(nn.Module):
    def __init__(self):
        super(ConvnextBase, self).__init__()
        self.model_name = 'convnext_base'
        self.pretrained = True

        self.trunk = timm.create_model(
            self.model_name,
            pretrained=self.pretrained,
        )
        self.trunk.head.fc = common_functions.Identity()
        self.embedder = nn.Linear(OUTPUT_SIZE, EMBEDDING_SIZE)
    
    def forward(self, x):
        x = self.trunk(x)
        x = self.embedder(x)
        return x

In [18]:
model = ConvnextBase()
model = model.to(device)

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_base_1k_224_ema.pth


Losses

In [19]:
from pytorch_metric_learning import losses, distances, regularizers


def get_arcfaceloss():
    distance = distances.CosineSimilarity()
    regularizer = regularizers.RegularFaceRegularizer()
    loss = losses.ArcFaceLoss(
        num_classes=2,
        embedding_size=EMBEDDING_SIZE,
        margin=28.6,
        scale=64,
        weight_regularizer=regularizer, 
        distance=distance
    )
    sampler = None
    mining_funcs = dict()

    return loss, sampler, mining_funcs

In [20]:
loss, sampler, mining_funcs = get_arcfaceloss()

loss = loss.to(device)

Optimizer

In [21]:
import torch
from torch import optim


def get_optimizer(net: nn.Module):
    return optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-5)

In [22]:
optimizer = get_optimizer(model)
loss_optimizer = get_optimizer(loss)

In [30]:
from tqdm import tqdm


def train(model, loss_fn, train_dataloader, device, epoch, optimizer, loss_optimizer):
    model.train()
    total_loss = 0
    for data, labels in tqdm(train_dataloader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        loss_optimizer.zero_grad()
        embeddings = model(data)
        loss = loss_fn(embeddings, labels)
        loss.backward()
        optimizer.step()
        loss_optimizer.step()
        total_loss += loss.detach().cpu().numpy()
    print(f'Epoch {epoch}: train_loss = {loss}')
    return total_loss

In [31]:
def test(model, loss_fn, test_dataloader, device, epoch):
    model.eval()
    with torch.no_grad():    
        total_loss = 0
        for data, labels in tqdm(test_dataloader):
            data, labels = data.to(device), labels.to(device)
            embeddings = model(data)
            loss = loss_fn(embeddings, labels)
            total_loss += loss.detach().cpu().numpy()
        print(f'Epoch {epoch}: test_loss = {loss}')
        return total_loss

In [32]:
import pytorch_metric_learning
from pytorch_metric_learning import testers


def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

In [33]:
def calculate_accuracy(train_dataset, test_dataset, model, accuracy_calculator):
    model.eval()
    train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
    test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print('Computing accuracy')
    print(f'test_Precision@1 = {accuracies["precision_at_1"]}')
    print(f'test_NMI = {accuracies["NMI"]}')
    print(f'test_AMI = {accuracies["AMI"]}')
    print(f'test_r_precision = {accuracies["r_precision"]}')
    print(f'test_mean_average_precision_at_r = {accuracies["mean_average_precision_at_r"]}')

In [34]:
accuracy_calculator = AccuracyCalculator(k='max_bin_count')

In [ ]:
class ModelCheckpoint():
    def __init__(self, mode='min'):
        self.mode=mode

        if mode=='min'

In [35]:
def train_model(model, loss, train_dataset, test_dataset, train_dataloader, test_dataloader, device, epochs, accuracy_calculator, optimizer, loss_optimizer):
    best_loss = np.inf
    for epoch in range(1, epochs + 1):
        train_loss = train(model, loss, train_dataloader, device, epoch, optimizer, loss_optimizer)
        val_loss = test(model, loss, val_dataloader, device, epoch)
        calculate_accuracy(train_dataset, val_dataset, model, accuracy_calculator)
        if val_loss < best_loss:
            best_loss = val_loss
            filename = f'best_{epoch}.pth'
            torch.save(model.state_dict(), filename)

In [36]:
train_model(model, loss, train_dataset, val_dataset, train_dataloader, val_dataloader, device, epochs, accuracy_calculator, optimizer, loss_optimizer)

100%|██████████| 24/24 [00:27<00:00,  1.13s/it]


Epoch 1: train_loss = 4.266952037811279


100%|██████████| 63/63 [00:01<00:00, 37.75it/s]


Epoch 1: test_loss = 33.10725402832031


100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


...

Epoch 10: train_loss = 0.006319019943475723


100%|██████████| 63/63 [00:01<00:00, 35.98it/s]


Epoch 10: test_loss = 24.968706130981445


100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


Computing accuracy
test_Precision@1 = 1.0
test_NMI = 0.895755222923823
test_AMI = 0.8944802104056611
test_r_precision = 0.9832766439909296
test_mean_average_precision_at_r = 0.9796742159376253
